## A Hands-on Workshop series in Machine Learning
### Session 5: More on Keras
#### Instructor: Aashita Kesarwani

Now that we know a bit more about neural nets, we will learn how to implement these new ideas with Keras. We will continue to use the Keras python library with TensorFlow backend. Keras provides a user-friendly interface to use TensorFlow and build the models quickly. Both Keras and TensorFlow are robust and powerful python libraries commonly used for deep learning.

There are two ways to build models in Keras, sequential and functional. 
* The sequential API allows us to create layer-by-layer models with multiple inputs and outputs, but is limited in its flexibility. 
* The functional API allows us to create models where we can connect any layer to any other layer, rather than only the layers immediately adjacent to it. It is useful for more advanced architectures where sequential API no longer suffices.

In this primer we will be focusing on the sequential API for ease and clarity.

For the purpose of reproducibility, we seed first.

In [1]:
from numpy.random import seed
seed(10)

from tensorflow import set_random_seed
set_random_seed(1)

We will use [Titanic dataset from Kaggle](https://www.kaggle.com/c/titanic), that we already work with in the earlier sessions. Let us first import the data into a pandas dataframe `df`.

In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd

path = 'data/'
df = pd.read_csv(path + 'titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Let us check the number of rows and columns in the dataset.

In [3]:
df.shape

(891, 12)

We will preprocess the dataset and extract our input features `X` and the target output `y` from the dataframe `df`:

In [4]:
import re

def get_Titles(df):
    df.Name = df.Name.apply(lambda name: re.findall("\w+[.]", name)[0].strip())
    df = df.rename(columns = {'Name': 'Title'})
    df.Title.replace({'Ms.': 'Miss.', 'Mlle.': 'Miss.', 'Dr.': 'Rare', 'Mme.': 'Mr.', 'Major.': 'Rare', 'Lady.': 'Rare', 'Sir.': 'Rare', 'Col.': 'Rare', 'Capt.': 'Rare', 'Countess.': 'Rare', 'Jonkheer.': 'Rare', 'Dona.': 'Rare', 'Don.': 'Rare', 'Rev.': 'Rare'}, inplace=True)
    return df

def fill_Age(df):
    df.Age = df.Age.fillna(df.groupby("Title").Age.transform("median"))
    return df

def get_Group_size(df):
    Ticket_counts = df.Ticket.value_counts()
    df['Ticket_counts'] = df.Ticket.apply(lambda x: Ticket_counts[x])
    df['Family_size'] = df['SibSp'] + df['Parch'] + 1
    df['Group_size'] = df[['Family_size', 'Ticket_counts']].max(axis=1)
    return df

def process_features(df):
    df.Sex = df.Sex.astype('category', ordered=False).cat.codes
    features_to_keep = ['Age', 'Fare', 'Group_size', 'Pclass', 'Sex']
    df = df[features_to_keep]
    return df

def process_data(df):
    df = df.copy()
    df = get_Titles(df)
    df = fill_Age(df)
    df = get_Group_size(df)
    df = process_features(df)
    medianFare = df['Fare'].median()
    df['Fare'] = df['Fare'].fillna(medianFare)
    return df

X = process_data(df)
y = df["Survived"].astype('category', ordered=False)

Now we want to split the data into training and testing sets. This is for model cross-validation, that is once our network is trained using training data, we want to measure its performance on the unseen validation data.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1, test_size=0.2)

### Constructing a simple network

We import the relevant functions from [Keras](https://keras.io/) that we plan to use in our network.

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras import losses



Using TensorFlow backend.


To build a squential, feedforward multilayer perceptron we need to start by constructing an appropriately formatted neural network. Let us first look at the input size, that is the number of features.

In [7]:
input_dim = X_train.shape[1] # size of input variables
print("Input size:", input_dim)

Input size: 5


So, the network will take in 5 input variables and output a single variable. We will first work out a simple example with one input layer, one hidden layer, and one output layer. Because we are taking in 5 inputs and deterimining one output, we want our first layer to have dimension 5 and our last to have dimension 1.

Let us construct a network with a 5-5-1 architecture, meaning that the input layer have 5 nodes, the hidden layers have 5 nodes, and the output layer has a single node for the binary classification - survived or not. 

We begin by creating a Sequential model using [Keras Sequential API](https://keras.io/getting-started/sequential-model-guide/).

In [8]:
model = Sequential()

We begin by creating a Sequential model, then using the [`.add()`](https://keras.io/getting-started/sequential-model-guide/) function to add the desired layers. For this example we want each layer to be dense, so each node in one layer is connected to every node in the next.

Some notes:
* We add layers to the model via the [`.add()`](https://keras.io/getting-started/sequential-model-guide/) method. 
* Keras API allows us to define various kinds of layers that are useful for more involved architectures, but for the Multi-layer Perceptrons, in which every node is connected to every other node in the consequents layers, we use [`Dense`](https://keras.io/layers/core/) layers. 
* The very first layer added to the model needs to be given the input shape (`input_shape`) or input dimension (`input_dim`).
* The other two properties we will specify for the Dense layers are
    * Number of units: We will use 5 nodes for the hidden layer
    * Activation function: We will use sigmoid activation
    
Let us add the first fully-connected (dense) layer with the sigmoid activation using [`.add()`](https://keras.io/getting-started/sequential-model-guide/) method with [`Dense`](https://keras.io/layers/core/) layer. This is the hidden layer with 5 nodes (or units).

In [9]:
model.add(Dense(units=5, input_dim=input_dim, activation="sigmoid")) # Hidden layer

Now, we add the output layer. 
* No need to specify the input size, Keras will automatically take it to be 25 - same as the output size of the previous layer. 
* We specify the the number of units for this layer to be 1 - to give use the prediction of whether the tumor is benign or malignant.

In [10]:
model.add(Dense(units=1, activation="sigmoid")) # Output layer

Let us check the structure of the network using the [`.summary()`](https://keras.io/models/about-keras-models/) function.

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 36
Trainable params: 36
Non-trainable params: 0
_________________________________________________________________


This seems to give us the desired structure. Keep in mind that in practice we want to optimize the number of nodes per hidden layer and number of hidden layers, while in this example the layers were chosen for explanatory purposes.

### Training and testing the model 

Now that we have defined our network architecture, we need to compile it first using [`compile`](https://keras.io/models/model/) before running it. There are a few keywords that we need to pass:
* `optimizer`: We will use the stochastic gradient descent menthod `SGD` for the optimizing the cost/loss function, that we studied earlier. We will set the learning rate (step size) to be `lr=0.001`. There are also a few variants of this method that we will learn and use in the next session.
* `loss`: This is nothing but the cost function. We want to build a model for binary classification and hence will use cross-entropy loss, as covered in the logistic classifier in the last session.
* `metrics`: We studied a lot of classification metrics in our last session. Here, we will simply use accuracy.


In [12]:
model.compile(optimizer=SGD(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])

Finally, we train the model using 10 rounds (epochs) over the entire training set.

In [13]:
model.fit(X_train, y_train, epochs=30, verbose=1); 

Epoch 1/30
712/712 [==============================] - 0s 676us/step - loss: 0.7636 - acc: 0.3778
Epoch 2/30
712/712 [==============================] - 0s 43us/step - loss: 0.7552 - acc: 0.3778
Epoch 3/30
712/712 [==============================] - 0s 49us/step - loss: 0.7463 - acc: 0.3904
Epoch 4/30
712/712 [==============================] - 0s 42us/step - loss: 0.7370 - acc: 0.4059
Epoch 5/30
712/712 [==============================] - 0s 42us/step - loss: 0.7269 - acc: 0.4185
Epoch 6/30
712/712 [==============================] - 0s 46us/step - loss: 0.7171 - acc: 0.4270
Epoch 7/30
712/712 [==============================] - 0s 42us/step - loss: 0.7059 - acc: 0.4691
Epoch 8/30
712/712 [==============================] - 0s 48us/step - loss: 0.6949 - acc: 0.5351
Epoch 9/30
712/712 [==============================] - 0s 42us/step - loss: 0.6847 - acc: 0.5590
Epoch 10/30
712/712 [==============================] - 0s 41us/step - loss: 0.6753 - acc: 0.5716
Epoch 11/30
712/712 [=================

We note that for each epoch the accuracy is generally increasing and the loss is generally decreasing. We now want to see how well it works on our validation set. 

In [14]:
# show the accuracy on the testing set
print("Evaluating on testing set...")
(loss, accuracy) = model.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Evaluating on testing set...
179/179 [==============================] - 0s 510us/step
[INFO] loss=0.6479, accuracy: 62.5698%


### Adding more layers:

Let us see what happens if we add an additional hidden layer:

In [15]:
model2 = Sequential()
model2.add(Dense(units=5, input_dim=input_dim, activation="sigmoid")) # 1st hidden layer
model2.add(Dense(units=8, activation="sigmoid")) # 2nd hidden layer
model2.add(Dense(units=1, activation="sigmoid")) # output layer
model2.compile(optimizer=SGD(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"])
model2.fit(X_train, y_train, epochs=30, verbose=1); 

# show the accuracy on the testing set
print("\n\nEvaluating on testing set...")
(loss, accuracy) = model2.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

Epoch 1/30
712/712 [==============================] - 0s 666us/step - loss: 0.6684 - acc: 0.6222
Epoch 2/30
712/712 [==============================] - 0s 43us/step - loss: 0.6682 - acc: 0.6222
Epoch 3/30
712/712 [==============================] - 0s 43us/step - loss: 0.6681 - acc: 0.6222
Epoch 4/30
712/712 [==============================] - 0s 40us/step - loss: 0.6680 - acc: 0.6222
Epoch 5/30
712/712 [==============================] - 0s 41us/step - loss: 0.6678 - acc: 0.6222
Epoch 6/30
712/712 [==============================] - 0s 40us/step - loss: 0.6677 - acc: 0.6222
Epoch 7/30
712/712 [==============================] - 0s 40us/step - loss: 0.6676 - acc: 0.6222
Epoch 8/30
712/712 [==============================] - 0s 39us/step - loss: 0.6675 - acc: 0.6222
Epoch 9/30
712/712 [==============================] - 0s 43us/step - loss: 0.6673 - acc: 0.6222
Epoch 10/30
712/712 [==============================] - 0s 40us/step - loss: 0.6672 - acc: 0.6222
Epoch 11/30
712/712 [=================

The loss function in the above network was not changing much and there was no change in the accuracy metric. It seems that the network was suffering from the vanishing gradient problem as we added more depth to our network, the weight updates were neglible.

### Using ReLU as an activation function:

To address the vanishing gradient in the above network, we replace the sigmoid activation function with ReLU, expect for the output layer. We also increased the learning rate in the gradient descent by tha factor of 10 by passing `SGD(lr=0.01)` instead of `SGD(lr=0.001)` for the optimizer.

In [16]:
model3 = Sequential()
model3.add(Dense(units=5, input_dim=input_dim, activation="relu")) 
model3.add(Dense(units=8, activation="relu"))
model3.add(Dense(units=1, activation="sigmoid")) 
model3.compile(optimizer=SGD(lr=0.01), loss="binary_crossentropy", metrics=["accuracy"])
model3.fit(X_train, y_train, epochs=30, verbose=1); 

# show the accuracy on the testing set
print("\n\nEvaluating on testing set...")
(loss, accuracy) = model3.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Epoch 1/30
712/712 [==============================] - 1s 711us/step - loss: 0.6928 - acc: 0.6236
Epoch 2/30
712/712 [==============================] - 0s 47us/step - loss: 0.6294 - acc: 0.6222
Epoch 3/30
712/712 [==============================] - 0s 42us/step - loss: 0.6071 - acc: 0.6222
Epoch 4/30
712/712 [==============================] - 0s 44us/step - loss: 0.5980 - acc: 0.6320
Epoch 5/30
712/712 [==============================] - 0s 45us/step - loss: 0.5888 - acc: 0.6826
Epoch 6/30
712/712 [==============================] - 0s 44us/step - loss: 0.5876 - acc: 0.7065
Epoch 7/30
712/712 [==============================] - 0s 44us/step - loss: 0.5829 - acc: 0.7205
Epoch 8/30
712/712 [==============================] - 0s 47us/step - loss: 0.5833 - acc: 0.7107
Epoch 9/30
712/712 [==============================] - 0s 46us/step - loss: 0.5824 - acc: 0.7177
Epoch 10/30
712/712 [==============================] - 0s 45us/step - loss: 0.5808 - acc: 0.7247
Epoch 11/30
712/712 [=================

This has indeed improved the network quite significantly. 

### Regularization

Lastly, we increase the complexity of the network by adding more nodes in each layer. We also use the RMSprop as the optimizer instead of gradient descent, as it often performs better for more complex networks.

In [17]:
from keras import regularizers
model4 = Sequential()
model4.add(Dense(units=25, input_dim=input_dim, activation="sigmoid", 
                 activity_regularizer=regularizers.l2(0.001))) 
model4.add(Dense(units=10, activation="relu",
                activity_regularizer=regularizers.l2(0.001))) 
model4.add(Dense(units=1, activation="sigmoid")) 
model4.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model4.fit(X_train, y_train, epochs=100, verbose=1); 

# show the accuracy on the testing set
print("\n\nEvaluating on testing set...")
(loss, accuracy) = model4.evaluate(X_valid, y_valid, batch_size=5, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))


Epoch 1/100
712/712 [==============================] - 1s 904us/step - loss: 1.0509 - acc: 0.6222
Epoch 2/100
712/712 [==============================] - 0s 53us/step - loss: 0.9386 - acc: 0.6222
Epoch 3/100
712/712 [==============================] - 0s 53us/step - loss: 0.8769 - acc: 0.6222
Epoch 4/100
712/712 [==============================] - 0s 56us/step - loss: 0.8385 - acc: 0.6924
Epoch 5/100
712/712 [==============================] - 0s 53us/step - loss: 0.8115 - acc: 0.6980
Epoch 6/100
712/712 [==============================] - 0s 55us/step - loss: 0.7869 - acc: 0.6938
Epoch 7/100
712/712 [==============================] - 0s 54us/step - loss: 0.7660 - acc: 0.6840
Epoch 8/100
712/712 [==============================] - 0s 55us/step - loss: 0.7498 - acc: 0.6910
Epoch 9/100
712/712 [==============================] - 0s 73us/step - loss: 0.7361 - acc: 0.6966
Epoch 10/100
712/712 [==============================] - 0s 60us/step - loss: 0.7240 - acc: 0.6966
Epoch 11/100
712/712 [======

**Note: The training example is too small to meaningfully learn the tuning of neural networks, and the primer is intended to give you a simple illustration for using Keras in your projects.**

#### Acknowledgment:
* The dataset used in this project is the [Breast Cancer Wisconsin (Diagnostic) Data Set](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data).